In [1]:
import sys
import os

toolkit_dir = os.path.abspath('./toolkit')
sys.path.append(toolkit_dir)

import numpy as np
import streamlit as st
from toolkit.adkp_queries import (
    query_project_summary,
    query_monthly_downloads_per_study
)
from toolkit.utils import get_data_from_snowflake
from toolkit.widgets import plot_download_sizes, plot_unique_users_trend

2024-08-28 16:31:20.051 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
# 1. Retrieve the data using your queries in queries.py
project_summary_df = get_data_from_snowflake(query_project_summary())

2024-08-28 16:31:20.553 
  command:

    streamlit run /Users/alinden/.pyenv/versions/3.10.9/envs/snowflake-streamlit-3.10.9/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-08-28 16:31:20.554 No runtime found, using MemoryCacheStorageManager


In [3]:
project_summary_df

,TOTAL_FILES,TOTAL_SIZE_IN_TB,STORAGE_COST_PER_YEAR
0,260520,811.82,229439.39


In [4]:
unique_users_df = get_data_from_snowflake(query_monthly_downloads_per_study())

In [5]:
unique_users_df

,STUDY,MONTH,NUMBER_OF_DOWNLOADS,NUMBER_OF_UNIQUE_USERS,GIB_DOWNLOADED
0,ROSMAP,2024-08-01,46712,137,99004.74
1,MIT_ROSMAP_Multiomics,2024-08-01,85959,50,133003.87
2,AMP-AD_DiverseCohorts,2024-08-01,33880,25,44063.30
3,MSBB,2024-08-01,3438,22,2798.56
4,rnaSeqReprocessing,2024-08-01,2351,19,19680.67
...,...,...,...,...,...
2416,APOEPSC,2022-01-01,2,1,0.00
2417,AD_CrossSpecies,2022-01-01,1,1,0.02
2418,ADMC_ADNI2-GO,2022-01-01,11,1,0.00
2419,AD-BXD,2022-01-01,1,1,0.00


In [6]:
total_files = project_summary_df.loc[0,'TOTAL_FILES']
total_data_volume = project_summary_df.loc[0, 'TOTAL_SIZE_IN_TB']

In [7]:
col1, col2, col3 = st.columns([1, 1, 5])
col1.metric("Total Number of Files", f"{total_files}")
col2.metric("Total Data Volume", f"{total_data_volume} TB")

DeltaGenerator()

In [8]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [15]:
grouped_df = (
    unique_users_df.groupby(["STUDY", "MONTH"])["NUMBER_OF_UNIQUE_USERS"]
    .sum()
    .reset_index()
)

In [36]:
grouped_df["MONTH"] = pd.to_datetime(grouped_df["MONTH"])

In [37]:
grouped_df.loc[grouped_df["MONTH"] == "2024-07-01"]

,STUDY,MONTH,NUMBER_OF_UNIQUE_USERS
45,ACT,2024-07-01,1
76,AD-BXD,2024-07-01,4
130,ADMC_ADNI2-GO,2024-07-01,1
187,ADMC_ADNI_UHawaiiGutMetabolites,2024-07-01,1
223,AD_CrossSpecies,2024-07-01,3
...,...,...,...
2242,iPSCMicroglia,2024-07-01,3
2258,miR155,2024-07-01,2
2321,rnaSeqReprocessing,2024-07-01,18
2387,snRNAseqAD_TREM2,2024-07-01,3


In [27]:
month = pd.to_datetime(grouped_df["MONTH"])

In [33]:
month.unique

<bound method Series.unique of 0      2022-01-01
1      2022-03-01
2      2022-04-01
3      2022-05-01
4      2022-06-01
          ...    
2416   2024-04-01
2417   2024-05-01
2418   2024-06-01
2419   2024-07-01
2420   2024-08-01
Name: MONTH, Length: 2421, dtype: datetime64[ns]>

In [40]:
recent_top_studies = (
    grouped_df.loc[grouped_df["MONTH"] == "2024-07-01"]
    .sort_values(by="NUMBER_OF_UNIQUE_USERS", ascending=False)
    .head(10)
)

In [41]:
fig = go.Figure()